# Tensorfow tf.Data and Dataset API을 활용하는 간단한 Data augmentation과정.

>* Wouter Bulten의 [imple and efficient data augmentations using the Tensorfow tf.Data and Dataset API](https://www.wouterbulten.nl/blog/tech/data-augmentation-using-tensorflow-data-dataset/) 글을 번역한 내용입니다.
>
>* [Tensorflow Documnet, tf.image](https://www.tensorflow.org/api_docs/python/tf/image)

<i><font color=gray>Summary: The tf.data API of Tensorflow is a great way to build a pipeline for sending data to the GPU. In this post I give a few examples of augmentations and how to implement them using this API.<font></i>

Tensorflow의 `tf.data` API는 data를 GPU로 보내는 pipeline을 구성하기 좋은 방법이다. 하지만 Data augmentation 과정을 만드는게(setting) 조금 어렵기도(tricky)하다. Tutorial에서 data augmentation popeline을 구성하는 방법을 알아보자.

## The table of contents

* 회전과 반전(Rotation and flipping)
* 색 반전(Color augmentations)
* 확대 및 축소(Zooming)
* 여러 과정을 함께사용(All augmentations combine)
* Full code example

## Some sample data

augmentation techniques을 실습하기 위해 DEMO DATA로 __CIFAR10__ dataset을 사용할 것이다. CIFAR10은 Tensorflow에서 쉽게 불러올 수 있다. 여기서는 먼저 8개의 예시용 이미지를 사용할 것이고, 원하는 데이터로 대체해도 상관없다.

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt

`keras.dataset`을 이용해서 cifar10을 바로 불러올 수 있다.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
x_train.shape

(50000, 32, 32, 3)

불러온 data를 `tf.dat.Dataset`형태로 불러오자. `tf.data`에 관한 내용은 [Tensorflow, 'Importing data'](https://www.tensorflow.org/guide/datasets)를 참고하면 좋다.

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_train[0:8] / 255).astype(np.float32))
print(dataset)

<TensorSliceDataset shapes: (32, 32, 3), types: tf.float32>


tf.data의 `repeat`과 `batch`를 적용하면 어떻게 output이 정해지는 지 실험해보자.

기본적으로 dataset을 출력하면 (32,32,3)이 나온다. 저장된 8장의 이미지 중, 1장을 불러온다고 볼 수 있다. 이 과정을 반복해주면 8장의 이미지를 모두 볼러오고 반복을 종료하게 된다.

In [5]:
for i in dataset:
    print(i.shape)

(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)


아래에서는 `repeat`을 적용해보자. 8장의 이미지를 불러오는 과정을 2번 반복하므로 16장의 이미지를 사용하는 것을 볼 수 있다.

In [6]:
for i in dataset.repeat(2):
    print(i.shape)

(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)
(32, 32, 3)


`batch`를 적용하게 되면, batch size 만큼 묶어서 반복하게 된다. 8장의 이미지를 batch_size=4로 묶었으므로 2개의 batch를 생성한다.

In [7]:
for i in dataset.batch(4):
    print(i.shape)

(4, 32, 32, 3)
(4, 32, 32, 3)


'map'은 dataset를 불러오면서 lambda로 정의한 함수를 통과시켜 결과로 출력해준다. 간단하게 이미지 앞에 차원을 추가하는 함수를 적용하면 다음과 같다.

아래의 augmentation 과정은 함수를 정의해서 map을 추가하는 방법을 사용할 것이다.

In [8]:
for i in dataset.map(lambda x : tf.expand_dims(x, axis=0)):
    print(i.shape)

(1, 32, 32, 3)
(1, 32, 32, 3)
(1, 32, 32, 3)
(1, 32, 32, 3)
(1, 32, 32, 3)
(1, 32, 32, 3)
(1, 32, 32, 3)
(1, 32, 32, 3)


만든 dataset의 image를 plot에 출력해보자.

In [ ]:
def plot_images(dataset, n_images, samples_per_image):
    '''
    dataset : Input dataset
    n_images : sample images number. In tutorial, n_images =8
    samples_per_image : repeat images. In tutorial, samples_per_image=10
    '''
    output = np.zeros((32*n_images, 32*samples_per_image, 3))
    
    row = 0
    
    for images in dataset.repeat(samples_per_image).batch(n_images):
        output[:, row*32:(row+1)*32] = np.vstack(images.numpy())
        row +=1
        
    plt.figure()
    plt.imshow(output)
    plt.show()

In [ ]:
#plot_images(dataset, 8,10)

<br>
<div align="center">
    <img src="https://d33wubrfki0l68.cloudfront.net/761b3558810672e58812008ee0ac8b24d55d6a1f/1e1d0/assets/images/deep-learning/tf_data_no_augmentation_cifar.png" width="500">
    <br>
    <i>Upsampled examples from the CIFAR dataset before any data augmentation was applied.</i>
</div>
<br>

## Implementing augmentations

Augmentation 과정을 적용하기 위해서는 'augmenter functions'을 만드는 방법이 좋다. 이 함수는 image(a tf.Tensor)을 입력받아 augmented image를 출력한다. Augmentation 과정(operation)을 각각의 함수로 정의하고 불러온 dataset에 적용해보자.

augmenter functions의 기본적인 틀은 다음과 같다.

In [ ]:
def augment(x: tf.Tensor) -> tf.Tensor:
    """Some augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = .... # augmentation here
    return x

여기서는 아래의 방법을 함수로 정의해볼 것이다.

* Orientation (flipping and rotation)
* Color augmentations (hue, saturation, brightness, contrast)
* Zooming

<i><font color=gray>Not all of these augmentations are necessarily applicable to CIFAR10; e.g. learning to detect flipped trucks is maybe not that beneficial for the task at hand. Nevertheless, I show them here as an example as they can be useful for tasks that are more orientation invariant. Of course, there are many more augmentations that could be useful, but most of them follow the same approach.<font></i>

## Rotation and flipping

### Rotation

가장 간단한 augmentation 방법은 image를 90도 회전시키는 것이다. 이 방법은 Tensorflow의 `rot90` function을 이용하면 된다. 무작위 각도로 생성하기 위해서는 각각의 이미지에 Random function을 동시에 적용하면 된다. Random function은 통계적인 방법을 사용해서 number을 생성할 것이다.

In [ ]:
def rotate(x: tf.Tensor) -> tf.Tensor:
    """Rotation augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    # Rotate 0, 90, 180, 270 degrees, If want -90 ~ 90, minval = -1, maxval=1.
    return tf.image.rot90(x, tf.random_uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

In [ ]:
#plot_images(dataset.map(lambda x : rotate(x)), 8, 10)

<br>
<div align="center">
    <img src="https://d33wubrfki0l68.cloudfront.net/c734a00105e0444b23373d380b3c737762d4a370/0071a/assets/images/deep-learning/tf_data_rotate.png" width="500">
    <br>
    <i>Random rotation applied to the dataset. Images of trucks are possibly not the best examples to demonstrate rotations on ;)</i>
</div>
<br>

### Flipping

Flipping도 간단한 augmentation 방법 중 하나다. Flipping은  `random_flip_left_right` and `random_flip_up_down`안에 random number 생성 과정이 내장되어 있기때문에 그대로 사용하면된다.

In [ ]:
def flip(x: tf.Tensor) -> tf.Tensor:
    """Flip augmentation

    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)

    return x

In [ ]:
#plot_images(dataset.map(lambda x : flip(x)), 8, 10)


<br>
<div align="center">
    <img src="https://d33wubrfki0l68.cloudfront.net/7dccce8bb1297fd1ce677ea768fbb547a8be9219/0a44d/assets/images/deep-learning/tf_data_flip.png" width="500">
    <br>
    <i>Random flips applied to the dataset.</i>
</div>
<br>

## Color augmentations

Color augmentations은 거의 모든 image leanring task에 적용되고 있다. Tensorflow에서는 4가지 color augmentations를 제공하고 있다.
* [hue](https://www.tensorflow.org/api_docs/python/tf/image/random_hue)
* [asturation](https://www.tensorflow.org/api_docs/python/tf/image/random_saturation)
* [brightness](https://www.tensorflow.org/api_docs/python/tf/image/random_brightness)
* [contrast](https://www.tensorflow.org/api_docs/python/tf/image/random_contrast)

이 함수는 range값만 필요하며, 각 이미지에 고유한(unique) augmentation을 적용하여 결과를 보여준다.

In [ ]:
def color(x: tf.Tensor) -> tf.Tensor:
    """Color augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

In [ ]:
#plot_images(dataset.map(lambda x : color(x)), 8, 8)

<br>
<div align="center">
    <img src="https://d33wubrfki0l68.cloudfront.net/c888f20a50373a8015eb29692c4f2e5851c6fbc4/54f16/assets/images/deep-learning/tf_data_color.png" width="500">
    <br>
    <i>Color augmentations applied to the dataset. These values can be tuned to fit the target dataset.</i>
</div>
<br>

## Zooming

Zooming은 object size에 작은 변화를 주면서 network를 견고하게(robust) 만들수 있는 매우좋은(powerful) 방법이다. 이 방법은 단일 함수로 완벽하게 작동하지 않기때문에(_there is no single function that performs this operation completely_) 실행하기 조금 어렵다. Tensorflow의 `crop_and_resize` 함수를 사용하면 이미지를 자르고(crop) 무작위(arbitrary) 크기로 조정(resize)해준다. 이 함수는 crop하기위한 normalized coordinate(좌표값인듯)을 포함하는 list of ‘crop boxes’값이 필요하다.

아래 함수에서는 numpy를 사용해서 20개의 crop box를 생성할 것이다. 각 box는 한번씩 `crop_and_resize` fucntion을 통과하고, 각 box로 crop된 image를 출력한다. 여기서는 총 20개의 이미지(potential cropped images)를 생성할 것이며,`tf.random_uniform`을 사용해서 이미지 하나를 무작위로 선택할 것이다. `tf.random_uniform`은 학습하는동안 새로운 number를 생성하므로 사용해도좋다.

학습 시에 data가 초기의 차원수(original dimension)을 유지할 수 있도록, `tf.cond`를 사용할 것이다. `tf.cond` 함수는 3개의 parameter(`predicate`(or condition), `ture_fn`(true function), `false`)가 필요하다. predicate는 `true_fn` or `false_fn`을 적용한 후에, true or false를 평가하는 operation이다. 여기서는 셀의 50%만 true를 return 할 수 있도록 random number generator을 사용할 것이다.`true_fn`은 cropping function을 적용하는 함수이며, `false_fn`은 원본 이미지를 retrun하는 identity function이다.

__Note:__ augmenter functions에는 random number 생성에 `np.random`을 쓰지 않는 것이 좋다. 이 함수는 TF data pipeline에서 한번만 생성되기때문에 모든 이미지에 대해 같은 augmentation이 적용된다.

In [ ]:
def zoom(x: tf.Tensor) -> tf.Tensor:
    """Zoom augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    
    #Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))
    
    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1,y1,x2,y2]
        
    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_ind=np.zeros(len(scales)), crop_size=(32, 32))
        # Return a random crop
        return crops[tf.random_uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]
    
    choice = tf.random_uniform(shape=[], minval=0, maxval=1, dtype=tf.float32)
    
    #Only apply cropiing 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))


In [ ]:
#plot_images(dataset.map(lambda x : zoom(x)), 8, 8)

<br>
<div align="center">
    <img src="https://d33wubrfki0l68.cloudfront.net/eab1fa7237a3da0a384a96b7d0b8bcc9424c0738/811e6/assets/images/deep-learning/tf_data_zoom.png" width="400">
    <br>
    <i>Random zooms applied to the dataset. The effect can be subtle.</i>
</div>
<br>

## Augmenting the Dataset

모든 함수를 single pipeline으로 결합해보자. `map function`을 사용하면, Tensorflow Dataset에 쉽게 적용할 수 있다. `map`은 함수를 받아서 augmented dataset로 돌려준다. 새로운 dataset을 평가하기위해서, 모든 요소들을 적용한 새로운 operation 함수를 정의할 것이다. map function을 연결하면(Chaining map functions) 새로운 data에 반복적으로 operation을 적용할 수 있다.

이 과정(operation)의 속도를 향상시키기위해 Tensorflow operation이 지원하는 병렬 연산을 수행할 것이다(we can execute them in parallel). tf.Data API에서 `num_parallel_calls`를 map 함수에 사용하면 된다. parameter의 수는 CPU가 연산 가능한 숫자로 적용해야 한다.

__Note:__ 몇 가지 operation은 image의 값이 normal range(0,1)를 벗어날 수도 있다. 따라서 해당 범위를 넘지않도록 clipping 함수인 `tf.clip_by_value`를 사용하는 방법을 추천한다.

In [ ]:
augmentations = [flip, color, zoom, rotate]

#Add the agumentations to the dataset
for f in augmentations:
    #Apply the augmentation, run 4 jobs in parallel.
    dataset = dataset.map(f, num_parallel_calls=4)
    
# Make sure that the values are still in [0, 1]
dataset = dataset.map(lambda x: tf.clip_by_value(x, 0, 1), num_parallel_calls=4)

In [ ]:
#plot_images(dataset, n_images=8, samples_per_image=10)

<br>
<div align="center">
    <img src="https://d33wubrfki0l68.cloudfront.net/7da02efb6033dd8323d7b565b7078eae6790fe3d/f32f8/assets/images/deep-learning/tf_data_augmented.png" width="400">
    <br>
    <i>All augmentations applied to the dataset. The frequency of the augmentations is high for illustration purposes. For training it is probably beneficial to make the augmentations a bit less extreme.</i>
</div>
<br>

Augmentation을 적용할 때, 특정 비율의 데이터에만 적용할 수도 있다. zoom 함수를 만들때 사용했던 `tf.cond`와 `tf.random_uniform`을 함께 사용하면 된다.

In [ ]:
for f in augmentations:
    # Apply an augmentation only in 25% of the cases.
    dataset = dataset.map(lambda x: tf.cond(tf.random_uniform([], 0, 1) > 0.75, lambda: f(x), lambda: x), num_parallel_calls=4)

In [ ]:
#plot_images(dataset, 8, 10)

<br>
<div align="center">
    <img src="https://d33wubrfki0l68.cloudfront.net/700a05fdf4b1e9abd3923d94f8c4529b8e38421f/41be1/assets/images/deep-learning/tf_data_partially_augmented.png" width="400">
    <br>
    <i>All augmentations applied to the dataset, now with a lower frequency.</i>
</div>
<br>

더 많은 augmentation 함수를 추가하고 싶으면, 함수를 정의하고 간단하게 agumentation list에 추가하면 된다. 

## Full code example used in this post

In [ ]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

def plot_images(dataset, n_images, samples_per_image):
    output = np.zeros((32 * n_images, 32 * samples_per_image, 3))

    row = 0
    for images in dataset.repeat(samples_per_image).batch(n_images):
        output[:, row*32:(row+1)*32] = np.vstack(images.numpy())
        row += 1

    plt.figure()
    plt.imshow(output)
    plt.show()

def flip(x: tf.Tensor) -> tf.Tensor:
    """Flip augmentation

    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)

    return x

def color(x: tf.Tensor) -> tf.Tensor:
    """Color augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

def rotate(x: tf.Tensor) -> tf.Tensor:
    """Rotation augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    return tf.image.rot90(x, tf.random_uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

def zoom(x: tf.Tensor) -> tf.Tensor:
    """Zoom augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    # Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_ind=np.zeros(len(scales)), crop_size=(32, 32))
        # Return a random crop
        return crops[tf.random_uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]


    choice = tf.random_uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

data = (x_train[0:8] / 255).astype(np.float32)
dataset = tf.data.Dataset.from_tensor_slices(data)

# Add augmentations
augmentations = [flip, color, zoom, rotate]

for f in augmentations:
    dataset = dataset.map(lambda x: tf.cond(tf.random_uniform([], 0, 1) > 0.75, lambda: f(x), lambda: x), num_parallel_calls=4)
dataset = dataset.map(lambda x: tf.clip_by_value(x, 0, 1))

plot_images(dataset, n_images=8, samples_per_image=10)